<a href="https://colab.research.google.com/github/Saaaaangmin/LLM/blob/main/LLM_LogSend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# LLM_LogSend.py
from fastapi import APIRouter, HTTPException
from pydantic import BaseModel
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os
import datetime

router = APIRouter()

MY_ACCOUNT = "osm9357@gmail.com"
MY_PASSWORD = "opbg ucrq hlgg nmcm"
TO_MAIL = "chlrkd0103@naver.com"
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

class LogRequest(BaseModel):
    log_date: str

def send_email(log_file_path):
    msg = MIMEMultipart()
    msg["Subject"] = "PosToCat 로그 파일입니다."
    msg["From"] = MY_ACCOUNT
    msg["To"] = TO_MAIL

    content = "안녕하세요.\nPosToCat Log파일을 전달드립니다.\n감사합니다."
    msg.attach(MIMEText(content, "plain"))

    with open(log_file_path, 'rb') as file:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(file.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename={os.path.basename(log_file_path)}')
        msg.attach(part)

    with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as smtp:
        smtp.login(MY_ACCOUNT, MY_PASSWORD)
        smtp.sendmail(MY_ACCOUNT, TO_MAIL, msg.as_string())

    return True

@router.post("/send-log")
async def send_log(request: LogRequest):
    log_date = request.log_date

    try:
        datetime.datetime.strptime(log_date, "%Y%m%d")
    except ValueError:
        raise HTTPException(status_code=400, detail="유효하지 않은 날짜 형식입니다. (YYYYMMDD 형식이어야 합니다.)")

    log_file_path = f"C:/NICElog/{log_date}.log"

    if not os.path.exists(log_file_path):
        raise HTTPException(status_code=404, detail=f"파일 '{log_file_path}'을 찾을 수 없습니다.")

    if send_email(log_file_path):
        return {"success": True, "message": "로그 파일이 성공적으로 전송되었습니다."}
    else:
        raise HTTPException(status_code=500, detail="로그 파일 전송 중 오류가 발생했습니다.")